In [2]:
import pandas as pd
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
captureSize = 35.8 #for tmb claculation
import seaborn as sns

In [3]:
dfm1 = pd.read_excel('./Freeman_raw/Supplementary_Table_4.xlsx',sheet_name = '8. Pre_batch_correction', index_col=0)
dfm2 = pd.read_excel('./Freeman_raw/Supplementary_Table_6.xlsx',sheet_name = '2. Pre_batch_correction', index_col=0)

In [4]:
dfm1.shape,dfm2.shape

((19820, 261), (19820, 196))

In [5]:
dfm = dfm1.join(dfm2)
df_mRNA_tpm = 2**dfm-1

In [6]:
df_ids = pd.read_excel('./Freeman_raw/Supplementary_Table_1.xlsx', sheet_name='12. Clinical and RNAseq IDs')
df_ids['is_in_FreeMan'] = df_ids.in_primary_154 | df_ids.in_secondary_180
df_ids = df_ids[['clinical_rna_id','rna_id','cohort','participant',  'is_in_FreeMan']]
df_ids = df_ids.set_index('clinical_rna_id')
df_ids.head(3)

,rna_id,cohort,participant,is_in_FreeMan
clinical_rna_id,,,,
Pt1,Pt1-baseline,Hugo,Pt1,True
Pt10,Pt10-baseline,Hugo,Pt10,True
Pt12,Pt12-baseline,Hugo,Pt12,True


In [7]:
df_mRNA_tpm.shape, df_ids.shape

((19820, 457), (457, 4))

# MGH
### add RECIST column, change `Response ` to `Freeman_response` 


In [13]:
mgh = pd.read_excel('./Freeman_raw/Supplementary_Table_1.xlsx', 
                               index_col=[0,1,2,3,4,5,6], 
                               sheet_name='6. MGH (RNA)', skiprows=[0]).reset_index()
mgh = mgh.rename(columns={'Response ':'Freeman_response'})

mgh = mgh.set_index('Sample_id')
mgh = mgh.join(df_ids)
mgh = mgh.reset_index()

mgh = mgh[mgh.Timing == 'Pre']

mgh_meta = pd.read_csv('./MGH_raw/samples.csv', index_col=0)
mgh['RECIST'] = mgh.Sample_id.map(mgh_meta.response)
mgh['tmb.dan'] = mgh.Sample_id.map(mgh_meta['tmb.dan'])
mgh['tmb.wang'] = mgh.Sample_id.map(mgh_meta['tmb.wang'])


print(mgh['RECIST'].value_counts())

cols = ['rna_id', 'Sample_id',  'RECIST', 'Patient', 'Age', 'Gender', 'Primary_tumor',
       'Overall_survival', 'Alive (Y=1, No=0)', 'Timing', 'Freeman_response',
       'Therapy', 'Prior_CTLA4',  'cohort', 'tmb.dan', 'tmb.wang', 'participant','is_in_FreeMan', ]
mgh = mgh[cols].set_index('rna_id')
mgh.to_csv('./Riaz_clinical.csv')


RECIST
NR    25
R     15
Name: count, dtype: int64


# Gide

### add RECIST column, change `Response ` to `Freeman_response`

In [14]:
def changeid(x):
    p, i, s = x.split('_')
    return '%s_%s_%s' % (i, p, s)
    
gide = pd.read_excel('./Freeman_raw/Supplementary_Table_1.xlsx', 
                     sheet_name='11. Gide et al. (RNA)',
                     skiprows=[0], index_col=[0,1,2,3,4,5,6]).reset_index()

gide = gide.rename(columns={'Response ':'Freeman_response'})

gide = gide.set_index('Sample_id')

gide = gide.join(df_ids)
gide = gide.reset_index()
gide.Sample_id = gide.Sample_id.apply(changeid)

gide_meta = pd.read_csv('./Gide_raw/meta.txt')
rnaseq_id = gide_meta.set_index('Sample_Name').Run.to_frame('rnaseq_id')
gide['rnaseq_id'] = gide.Sample_id.map(rnaseq_id.rnaseq_id)

gide = gide[gide.Timing == 'Pre']

gide_meta2 = pd.read_csv('./Gide_raw/samples.csv',index_col=0)
gide['RECIST'] = gide.rnaseq_id.map(gide_meta2.response)
gide['Age'] = gide.rnaseq_id.map(gide_meta2.age)
gide['Gender'] = gide.rnaseq_id.map(gide_meta2.gender)
print(gide['RECIST'].value_counts())
cols = ['rna_id', 'Sample_id',  'RECIST', 'Patient', 'Age', 'Gender', 'Primary_tumor',
       'Overall_survival', 'Alive (Y=1, No=0)', 'Timing', 'Freeman_response',
       'Therapy', 'Prior_CTLA4',  'cohort', 'rnaseq_id', 'participant','is_in_FreeMan', ]
gide = gide[cols].set_index('rna_id')
gide.to_csv('./Gide_clinical.csv')

RECIST
PR    26
PD    22
CR    14
SD    11
Name: count, dtype: int64


# Liu
### add RECIST column, change `Response ` to `Freeman_response`


In [15]:

liu = pd.read_excel('./Freeman_raw/Supplementary_Table_1.xlsx', 
                               index_col=[0,1,2,3,4,5,6], 
                               sheet_name='10. Liu et al. (RNA)', skiprows=[0]).reset_index()
liu = liu.rename(columns={'Response ':'Freeman_response'})

liu = liu.set_index('Sample_id')
liu = liu.join(df_ids)
liu = liu.reset_index()

liu = liu[liu.Timing == 'Pre']
liu.Patient = liu.Patient.apply(lambda x:x.split('_')[0])

liu_meta = pd.read_excel('./Liu_raw/41591_2019_654_MOESM4_ESM.xlsx', index_col=0, skiprows=[0,1]).iloc[:144]
liu['RECIST'] = liu.Patient.map(liu_meta.BR)

liu['Primary_tumor'] = liu.Patient.map(liu_meta['biopsy site'])
liu['Gender'] = liu.Patient.map(liu_meta['gender (Male=1, Female=0)'].map({1:'M', 0:'F'}))



VC_NONSYNONYMOUS_LIST = ['Frame_Shift_Del',
                         'Frame_Shift_Ins',
                         'In_Frame_Del',
                         'In_Frame_Ins',
                         'Missense_Mutation',
                         'Nonsense_Mutation',
                         'Splice_Site',
                         'Nonstop_Mutation',
                         'Splice_Region']

truncate_mut_LIST = ['Nonsense_Mutation', 'Frame_Shift_Ins','Frame_Shift_Del','Splice_Site']
nontruncate_mut_LIST = ['Missense_Mutation','In_Frame_Del', 'In_Frame_Ins','Nonstop_Mutation']

mut = pd.read_csv('./Liu_raw/all_mutation.csv', sep=',')

mut['nansynonymous_mut_flag'] = mut.Variant_Classification.isin(VC_NONSYNONYMOUS_LIST)
mut['truncate_mut_flag'] = mut.Variant_Classification.isin(truncate_mut_LIST)
mut['nontruncate_mut_flag'] = mut.Variant_Classification.isin(nontruncate_mut_LIST)


I_calculated_TMB1 = mut.groupby('Patient').nansynonymous_mut_flag.sum()
I_calculated_TMB2 = mut.groupby('Patient').apply(lambda x:x.truncate_mut_flag.sum()*2+x.nontruncate_mut_flag.sum())
#for tmb claculation
tmb1 = np.log2(I_calculated_TMB1/captureSize)
tmb2 = np.log2(I_calculated_TMB2/captureSize)

# tmb1.hist(bins=20)
# tmb2.hist(bins=20)

tmb = tmb1.to_frame('tmb.dan').join(tmb2.to_frame(name = 'tmb.wang'))

liu['tmb.dan'] = liu.Patient.map(tmb['tmb.dan'])
liu['tmb.wang'] = liu.Patient.map(tmb['tmb.wang'])


cols = ['rna_id', 'Sample_id',  'RECIST', 'Patient', 'Age', 'Gender', 'Primary_tumor',
       'Overall_survival', 'Alive (Y=1, No=0)', 'Timing', 'Freeman_response',
       'Therapy', 'Prior_CTLA4',  'cohort', 'tmb.dan', 'tmb.wang', 'participant','is_in_FreeMan', ]
liu = liu[cols].set_index('rna_id')
liu.to_csv('./Liu_clinical.csv')


# Riaz
### add RECIST column, change `Response ` to `Freeman_response`

In [16]:
def changeid(x):
    x = x.replace('_p', '_P')
    return x
    
riaz = pd.read_excel('./Freeman_raw/Supplementary_Table_1.xlsx', 
                               index_col=[0,1,2,3,4,5,6], 
                               sheet_name='8. Riaz et al. (RNA)', skiprows=[0]).reset_index()
riaz = riaz.rename(columns={'Response ':'Freeman_response'})
riaz.Sample_id = riaz.Sample_id.apply(changeid)

riaz = riaz.set_index('Sample_id')
riaz = riaz.join(df_ids)
riaz = riaz.reset_index()

riaz = riaz[riaz.Timing == 'Pre']

riaz_meta2 = pd.read_csv('./Riaz_raw/samples.csv',index_col=0)
riaz_meta2.index = riaz_meta2.index.map(lambda x:x.split('-')[0])
riaz['RECIST'] = riaz.rna_id.map(riaz_meta2.response)


VC_NONSYNONYMOUS_LIST = [
                         'missense_variant',
                         'stop_gained',
                         'splice_donor_variant&intron_variant ',
                         'splice_acceptor_variant&intron_variant',
                         'stop_lost',
                         'start_lost&splice_region_variant',
                         'missense_variant&splice_region_variant']

truncate_mut_LIST = ['stop_gained', 'splice_acceptor_variant&intron_variant', 'splice_donor_variant&intron_variant ']
nontruncate_mut_LIST = ['missense_variant','stop_lost']

mut = pd.read_excel('./Riaz_raw/1-s2.0-S0092867417311224-mmc3.xlsx', skiprows=[0,1,2])
mut_series = mut['Variant Classification']

mut['nansynonymous_mut_flag'] = mut_series.isin(VC_NONSYNONYMOUS_LIST)
mut['truncate_mut_flag'] = mut_series.isin(truncate_mut_LIST)
mut['nontruncate_mut_flag'] = mut_series.isin(nontruncate_mut_LIST)

I_calculated_TMB1 = mut.groupby('Patient').nansynonymous_mut_flag.sum()
I_calculated_TMB2 = mut.groupby('Patient').apply(lambda x:x.truncate_mut_flag.sum()*2+x.nontruncate_mut_flag.sum())

#for tmb claculation
tmb1 = np.log2(I_calculated_TMB1/captureSize)
tmb2 = np.log2(I_calculated_TMB2/captureSize)

# tmb1.hist(bins=20)
# tmb2.hist(bins=20)

tmb = tmb1.to_frame('tmb.dan').join(tmb2.to_frame(name = 'tmb.wang'))

riaz['tmb.dan'] = riaz.Patient.map(tmb['tmb.dan'])
riaz['tmb.wang'] = riaz.Patient.map(tmb['tmb.wang'])
print(riaz['RECIST'].value_counts())

cols = ['rna_id', 'Sample_id',  'RECIST', 'Patient', 'Age', 'Gender', 'Primary_tumor',
       'Overall_survival', 'Alive (Y=1, No=0)', 'Timing', 'Freeman_response',
       'Therapy', 'Prior_CTLA4',  'cohort', 'tmb.dan', 'tmb.wang', 'participant','is_in_FreeMan', ]
riaz = riaz[cols].set_index('rna_id')
riaz.to_csv('./Riaz_clinical.csv')

RECIST
PD    23
SD    16
PR     7
CR     3
Name: count, dtype: int64


# Hugo
### add RECIST column, change `Response ` to `Freeman_response`

In [17]:
hugo = pd.read_excel('./Freeman_raw/Supplementary_Table_1.xlsx', 
                               index_col=[0,1,2,3,4,5,6], 
                               sheet_name='9. Hugo et al. (RNA)', skiprows=[0]).reset_index()
hugo = hugo.rename(columns={'Response ':'Freeman_response'})

hugo = hugo.set_index('Sample_id')
hugo = hugo.join(df_ids)
hugo = hugo.reset_index()
hugo = hugo[hugo.Timing == 'Pre']


hugo_meta2 = pd.read_csv('./Hugo_raw/samples.csv',index_col=0)
hugo['RECIST'] = hugo.Patient.map(hugo_meta2.response).map({'Progressive Disease':'PD', 
                                                            'Partial Response':'PR', 'Complete Response':'CR'})
print(hugo['RECIST'].value_counts())


VC_NONSYNONYMOUS_LIST = ['Frame_Shift_Del',
                         'Frame_Shift_Ins',
                         'In_Frame_Del',
                         'In_Frame_Ins',
                         'Missense_Mutation',
                         'Nonsense_Mutation',
                         'Splice_Site',
                         'Nonstop_Mutation',
                         'Splice_Region']

truncate_mut_LIST = ['Nonsense_Mutation', 'Frame_Shift_Ins','Frame_Shift_Del','Splice_Site']
nontruncate_mut_LIST = ['Missense_Mutation','In_Frame_Del', 'In_Frame_Ins','Nonstop_Mutation']

mut = pd.read_csv('./Hugo_raw/data_mutations.txt', sep='\t')

mut['nansynonymous_mut_flag'] = mut.Variant_Classification.isin(VC_NONSYNONYMOUS_LIST)
mut['truncate_mut_flag'] = mut.Variant_Classification.isin(truncate_mut_LIST)
mut['nontruncate_mut_flag'] = mut.Variant_Classification.isin(nontruncate_mut_LIST)


I_calculated_TMB1 = mut.groupby('Tumor_Sample_Barcode').nansynonymous_mut_flag.sum()
I_calculated_TMB2 = mut.groupby('Tumor_Sample_Barcode').apply(lambda x:x.truncate_mut_flag.sum()*2+x.nontruncate_mut_flag.sum())
#for tmb claculation
tmb1 = np.log2(I_calculated_TMB1/captureSize)
tmb2 = np.log2(I_calculated_TMB2/captureSize)

# tmb1.hist(bins=20)
# tmb2.hist(bins=20)

tmb = tmb1.to_frame('tmb.dan').join(tmb2.to_frame(name = 'tmb.wang'))

hugo['tmb.dan'] = hugo.Patient.map(tmb['tmb.dan'])
hugo['tmb.wang'] = hugo.Patient.map(tmb['tmb.wang'])

cols = ['rna_id', 'Sample_id',  'RECIST', 'Patient', 'Age', 'Gender', 'Primary_tumor',
       'Overall_survival', 'Alive (Y=1, No=0)', 'Timing', 'Freeman_response',
       'Therapy', 'Prior_CTLA4',  'cohort', 'tmb.dan', 'tmb.wang', 'participant','is_in_FreeMan', ]
hugo = hugo[cols].set_index('rna_id')
hugo.to_csv('./Hugo_clinical.csv')

RECIST
PD    12
PR    10
CR     4
Name: count, dtype: int64


# Allen
### add RECIST column, change `Response ` to `Freeman_response`

In [18]:
allen = pd.read_excel('./Freeman_raw/Supplementary_Table_1.xlsx', 
                               index_col=[0,1,2,3,4,5,6], 
                               sheet_name='7. Van Allen et al. (RNA)', skiprows=[0]).reset_index()
allen = allen.rename(columns={'Response ':'Freeman_response'})
allen.cohort = 'Allen'
allen = allen.set_index('Sample_id')
allen = allen.join(df_ids)
allen = allen.reset_index()
allen = allen[allen.Timing == 'Pre']


allen_meta2 = pd.read_csv('./Allen_raw/samples.csv',index_col=0)
allen['RECIST'] = allen.rna_id.map(allen_meta2.RECIST)
allen['tmb.dan'] = allen.rna_id.map(allen_meta2['tmb.dan'])
allen['tmb.wang'] = allen.rna_id.map(allen_meta2['tmb.wang'])
print(allen['RECIST'].value_counts())
cols = ['rna_id', 'Sample_id',  'RECIST', 'Patient', 'Age', 'Gender', 'Primary_tumor',
       'Overall_survival', 'Alive (Y=1, No=0)', 'Timing', 'Freeman_response',
       'Therapy', 'Prior_CTLA4',  'cohort', 'tmb.dan', 'tmb.wang', 'participant','is_in_FreeMan', ]
allen = allen[cols].set_index('rna_id')
allen.to_csv('./Allen_clinical.csv')

RECIST
PD    25
SD     6
PR     5
CR     2
X      1
Name: count, dtype: int64


In [19]:
dfp = pd.concat([hugo, liu, mgh, riaz, allen, gide])
dfp = dfp[dfp.is_in_FreeMan]

In [20]:
dfp.to_excel('./model_data/v2.samples.xlsx')

In [21]:
dfp.groupby(['cohort','Freeman_response']).size().unstack()

Freeman_response,NR,R
cohort,,
Gide,27,46
Hugo,12,14
Liu,66,41
MGH,24,14
Riaz,41,10
Van Allen,26,13


In [22]:
df_mRNA_tpm_v2 = df_mRNA_tpm[dfp.index]
cols = df_mRNA_tpm_v2.index

gene_id_name_map = pd.read_pickle('/n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/gene_id_name.map')
gene_id_name_map.index = gene_id_name_map.index.map(lambda x:x.split('.')[0])
gene_id_name_map = gene_id_name_map.drop_duplicates()
gene_name_type_map = pd.read_pickle('/n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/gene_name_type.map')

df_mRNA_tpm_v2.index = df_mRNA_tpm_v2.index.map(gene_id_name_map)
df_mRNA_tpm_v2 = df_mRNA_tpm_v2.loc[df_mRNA_tpm_v2.index.dropna()]

df_mRNA_tpm_v2 = df_mRNA_tpm_v2[df_mRNA_tpm_v2.index.map(gene_name_type_map) =='protein_coding']

In [23]:
df_mRNA_tpm_v2.to_excel('./model_data/v2.tpm.xlsx')

In [24]:
df_mRNA_tpm_v2.head(3)

,Pt1-baseline,Pt2-baseline,Pt4-baseline,Pt5-baseline,Pt6-baseline,Pt7-baseline,Pt8-baseline,Pt9-baseline,Pt10-baseline,Pt12-baseline,...,53_PD1_PRE,54_PD1_PRE,5_PD1_PRE,6_PD1_PRE,6_ipiPD1_PRE,7_PD1_PRE,7_ipiPD1_PRE,8_PD1_PRE,8_ipiPD1_PRE,9_PD1_PRE
OR4F5,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
SAMD11,1.32,0.72,0.11,0.22,0.07,0.72,0.18,0.61,6.82,0.74,...,2.14,0.13,0.11,0.08,0.04,0.14,0.01,0.00,0.02,0.37
NOC2L,93.47,20.59,23.66,50.98,32.49,45.74,48.76,44.25,67.27,78.88,...,21.11,28.76,9.52,19.81,9.77,25.94,2.47,0.78,23.02,19.45


In [25]:
dfp.head(3)

,Sample_id,RECIST,Patient,Age,Gender,Primary_tumor,Overall_survival,"Alive (Y=1, No=0)",Timing,Freeman_response,Therapy,Prior_CTLA4,cohort,tmb.dan,tmb.wang,participant,is_in_FreeMan,rnaseq_id
rna_id,,,,,,,,,,,,,,,,,,
Pt1-baseline,Pt1,PD,Pt1,66.0,F,Skin,607.0,0.0,Pre,NR,PD1,0.0,Hugo,5.722283,5.792309,Pt1,True,NaN
Pt2-baseline,Pt2,PR,Pt2,55.0,M,Lung,927.0,1.0,Pre,R,PD1,0.0,Hugo,5.840224,5.929548,Pt2,True,NaN
Pt4-baseline,Pt4,PR,Pt4,62.0,M,Adrenal,948.0,1.0,Pre,R,PD1,0.0,Hugo,6.841279,6.937131,Pt4,True,NaN
